In [ ]:
# Session identifier, do not remove or modify.
from yields.session import Session
session = Session("bcb237a6-2c60-4bab-8ca3-398918d3be3c")

# Calculate training data specific quantities

In this notebook, you shall load the training data on which a given model has been developed and calculate and store some quantities from this train set that might be retrieved in a later stage to assess to potential changes of the application data as compared to this training data. Stability testing is an important element of any model risk assessment, because an unstable test data, one that is not representative of the training data, may cause a variety of problems (e.g. bias) and pose significant model risk. 

By calculating all trainingdata related statistics in here, you will not load the training data into memory at validation time (i.e. when you assess the model performance on the application data). This is considered a best practice because it reduces not only the memory overhead but also avoids so-called data leakage from the trainset into the application dataset.

As a use case we will consider the following assessments and the ingredients we need:

| Stability metric | Type of variable | What we need
| :- | :- | :-
| Population Stability Index (PSI) | Categorical | For the categorical variable of interest, the proportion per category (i.e. the discrete distribution)
| Kolmogorov-Smirnov test | Continuous | The continuous distribution as given by the samples in the trainingsdataset
| Discriminatory power deterioration | n/a | Calculate the Area-under-the-ROC-curve for the model applied on the trainingsdataset.

Once again, at the time of model assessment on the application data, these numbers will be used to be compared against this test data to assess potential model risk.


In [ ]:
# load yields libraries


# standard python libraries
import json, os
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.core.display import HTML
import sklearn.metrics as sk_metrics
import scipy.stats as stats

# plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from yields.providers.filesystem import ProviderFactoryFilesystem #To write into filesystem
from yields.providers.hdfs import ProviderFactoryHdfs #To write into HDFS

#misc
import datetime


In [ ]:
#Aliases of inputs, outputs, parameters
session.io.inputs, session.io.outputs, session.io.parameters

In [ ]:
input_artifact_ds = session.io.input('ds')
file_handler = input_artifact_ds.file(input_artifact_ds.files[0]).open()
raw_data = pd.read_csv(file_handler)

In [ ]:
input_artifact_dmm_ds = session.io.input('dmm_ds')
file_handler = input_artifact_dmm_ds.file(input_artifact_dmm_ds.files[0]).open()
schema = json.load(file_handler)

In [ ]:
target, prediction, continuous_inputs, categorical_inputs = schema['target'], schema['prediction'] ,  schema['continuous_inputs'] , schema['categorical_inputs']
print('TARGET VARIABLE: ', target)
print('MODEL PREDICTION VARIABLE: ', prediction)
print('CONTINUOUS inputs: ', continuous_inputs)
print('COUNTABLE inputs: ', categorical_inputs)

In [ ]:
#initialize an empty dictionary

keys = ['Categorical_training', 'Continuous_training' , 'AUC_training']
training_facts = {'Categorical_training' : {}, 'Continuous_training' : {} , 'AUC_training' : {}}


In [ ]:
for var in categorical_inputs:

    #Also, display the proportion of observations per category for each variable
    x = raw_data[var]
    x = x[~x.isna()]
    
    var_df = pd.DataFrame(columns = ['Category' , 'Number observations' , 'Proportion (%)'])
    [uni, counts] = np.unique(x,return_counts = True)
    var_df['Category'] = uni
    var_df['Number observations'] = counts
    
    prop = counts / len(x)
    var_df['Proportion (%)'] = prop

    dict_tmp = {}
    
    for k in range(len(uni)):     
    
        dict_tmp.update({int(uni[k]) : prop[k]})   
        
    training_facts['Categorical_training'].update({var : dict_tmp})
            
    display(var_df.set_index('Category').style.set_caption(f'Overview for <b>{var}</b>').format({'Proportion (%)' : "{:.2%}"}))


In [ ]:
for var in continuous_inputs:

    x = raw_data[var]
    x = x[~x.isna()].tolist()

    my_dict = dict() 
    
    for index,value in enumerate(x):
        
        my_dict[index] = value
    
    training_facts['Continuous_training'].update({var : my_dict})

    

In [ ]:
import sklearn.metrics as sk_metrics

auc_score = sk_metrics.roc_auc_score(raw_data[target], raw_data[prediction])

training_facts['AUC_training'] = auc_score

In [ ]:
# class NpEncoder(json.JSONEncoder):
#     def default(self, obj):
#         if isinstance(obj, np.integer):
#             return int(obj)
#         if isinstance(obj, np.floating):
#             return float(obj)
#         if isinstance(obj, np.ndarray):
#             return obj.tolist()
#         if isinstance(obj, pd.core.series.Series):
#             return obj.tolist()
#         return super(NpEncoder, self).default(obj)


In [ ]:
# create output artifact
output_artifact = session.io.output('training_metrics')
artifact_output_version = output_artifact.file("training_dataset_metrics.json",ProviderFactoryFilesystem())
json_data = json.dumps(training_facts)
artifact_output_version.write(json_data.encode())

#name of saved file. Will appear like this in the UI.
file_name = "training_dataset_metrics.json"

output_artifact.save()
